# Project 1 – Charles – Metrics

In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib
import seaborn as sns

In [2]:
data = pd.read_csv("Cleaned_Data.csv", low_memory=False)
df = data.sort_values(by=['distinct_id', 'time'], axis=0, ascending=True)
df['ae_session_length'] = df['ae_session_length'].fillna('-1')
df['ae_session_length'] = df['ae_session_length'].str.replace(',', '')
df['ae_session_length'] = df['ae_session_length'].astype(float)
df['time'] = df['time'] = pd.to_datetime(df['time'])
df

,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,...,ios_lib_version,ae_first_app_open_date,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown
83729,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-02-13 15:52:50,-1.0,Mumbai,Maharashtra,IN,1,1,NaN,...,NaN,NaN,NaN,NaN,INTL,0,0,0,0,0
62403,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-03-08 20:43:54,-1.0,Ahmedabad,Gujarat,IN,1,1,NaN,...,NaN,NaN,NaN,NaN,INTL,0,0,0,0,0
84492,AppOpen,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:32,-1.0,Palo Alto,California,US,1,0,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
84494,Visit Signup/Login Page,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:35,-1.0,Palo Alto,California,US,1,1,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
84495,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:49:02,30.1,Palo Alto,California,US,1,1,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101267,Feed - Scroll Down,xbarbarazhong@gmail.com_6704,2019-12-12 14:29:23,-1.0,Portland,Oregon,US,1,0,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
101269,Profile - Change Section,xbarbarazhong@gmail.com_6704,2019-12-12 14:29:28,-1.0,Portland,Oregon,US,1,0,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
101270,$ae_session,xbarbarazhong@gmail.com_6704,2019-12-12 14:30:54,190.4,Portland,Oregon,US,1,1,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
100331,AppOpen,yoon.s.jeong@gmail.com_6485,2019-10-09 11:12:29,-1.0,Chicago,Illinois,US,1,0,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0


In [4]:
def add_session_id(df):
    df['session_id'] = -1
    sessions = df[['name_x', 'distinct_id', 'time', 'ae_session_length']].loc[df['name_x'] == '$ae_session']
    sessions['start'] = sessions['time'] - np.array([pd.Timedelta(seconds=x) for x in sessions['ae_session_length']])
    curr_id = 1
    for index, row in sessions.iterrows():
        df.loc[(df['time'] <= row['time']) & (df['time'] >= row['start']) & (df['distinct_id'] == row['distinct_id']), 'session_id'] = curr_id
        curr_id += 1

add_session_id(df)
df
    

,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,...,ae_first_app_open_date,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown,session_id
83729,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-02-13 15:52:50,-1.0,Mumbai,Maharashtra,IN,1,1,NaN,...,NaN,NaN,NaN,INTL,0,0,0,0,0,-1
62403,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-03-08 20:43:54,-1.0,Ahmedabad,Gujarat,IN,1,1,NaN,...,NaN,NaN,NaN,INTL,0,0,0,0,0,-1
84492,AppOpen,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:32,-1.0,Palo Alto,California,US,1,0,NaN,...,NaN,NaN,NaN,US,1,0,0,0,0,1
84494,Visit Signup/Login Page,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:35,-1.0,Palo Alto,California,US,1,1,NaN,...,NaN,NaN,NaN,US,1,0,0,0,0,1
84495,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:49:02,30.1,Palo Alto,California,US,1,1,NaN,...,NaN,NaN,NaN,US,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101267,Feed - Scroll Down,xbarbarazhong@gmail.com_6704,2019-12-12 14:29:23,-1.0,Portland,Oregon,US,1,0,NaN,...,NaN,NaN,NaN,US,1,0,0,0,0,29813
101269,Profile - Change Section,xbarbarazhong@gmail.com_6704,2019-12-12 14:29:28,-1.0,Portland,Oregon,US,1,0,NaN,...,NaN,NaN,NaN,US,1,0,0,0,0,29813
101270,$ae_session,xbarbarazhong@gmail.com_6704,2019-12-12 14:30:54,190.4,Portland,Oregon,US,1,1,NaN,...,NaN,NaN,NaN,US,1,0,0,0,0,29813
100331,AppOpen,yoon.s.jeong@gmail.com_6485,2019-10-09 11:12:29,-1.0,Chicago,Illinois,US,1,0,NaN,...,NaN,NaN,NaN,US,1,0,0,0,0,29814


In [5]:
df.loc[df['session_id'] == -1]

,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,...,ae_first_app_open_date,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown,session_id
83729,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-02-13 15:52:50,-1.0,Mumbai,Maharashtra,IN,1,1,NaN,...,NaN,NaN,NaN,INTL,0,0,0,0,0,-1
62403,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-03-08 20:43:54,-1.0,Ahmedabad,Gujarat,IN,1,1,NaN,...,NaN,NaN,NaN,INTL,0,0,0,0,0,-1
78804,AppOpen,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-24 09:49:49,-1.0,Los Altos,California,US,1,0,NaN,...,NaN,NaN,NaN,US,1,0,0,0,0,-1
55831,AppOpen,001210DC-54C8-43AD-B295-148F47818391,2019-08-04 16:28:25,-1.0,Austin,Texas,US,1,1,NaN,...,NaN,NaN,NaN,US,1,0,0,0,0,-1
55867,Permissions for Locations,001210DC-54C8-43AD-B295-148F47818391,2019-08-04 16:29:32,-1.0,Austin,Texas,US,1,0,NaN,...,NaN,NaN,NaN,US,1,0,0,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60525,AppOpen,willwojt@gmail.com_6868,2020-02-03 12:19:01,-1.0,Washington,District of Columbia,US,1,1,NaN,...,NaN,NaN,NaN,US,1,0,0,0,0,-1
81763,$ae_updated,willwojt@gmail.com_6868,2020-02-21 06:01:30,-1.0,Washington,District of Columbia,US,1,1,NaN,...,NaN,NaN,NaN,US,1,0,0,0,0,-1
92278,$ae_updated,wolphramite@gmail.com_291,2019-08-20 01:42:41,-1.0,Washington,District of Columbia,US,1,0,NaN,...,NaN,NaN,NaN,US,1,0,0,0,0,-1
94245,$campaign_delivery,wolphramite@gmail.com_291,2019-10-31 07:28:16,-1.0,NaN,NaN,NaN,1,0,NaN,...,NaN,NaN,NaN,NaN,2,0,0,0,0,-1


In [100]:
#Pseudo code to assign session IDs
#def add_session_id(df):
    current_session_id = 1
    session_end = None
    session_duration = None
    distinct_id = None
    for index, row in df.iterrows():
        event_type = row['name_x']
        if event_type == '$ae_session':
            session_end = row['time']
            session_duration = row['ae_session_length']
            distinct_id = row['distinct_id']
            row['session_id'] = current_session_id
        else:    
            time_difference = (session_end - row['time']).total_seconds()
            if distinct_id == row['distinct_id'] and time_difference <= session_duration + 1:
                row['session_id'] = current_session_id
            else:
                current_session_id += 1
    return df

In [6]:
df.to_csv("Cleaned_Dataset_With_ID.csv")